# Plotting nuclear waste heat over time

First, import the necessary packages and define the timeframe and heat models of interest. 
Also let's make a convenience function to format our plots.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import ogstools.repoheat as repo

repo.units.time = "yrs"
repo.units.power = "kW"
time = np.geomspace(1, 1e6, num=100)
models = [m for m in repo.models if "2012" in m.name or "2020" in m.name]
ls = ["-", "--", "-.", ":", (0, (1, 10))]


def format_ax(ax: plt.Axes):
    ax.set_xlabel(f"time [{repo.units.time}]")
    ax.set_ylabel(f"heat [{repo.units.power }]")
    ax.grid(True, which="major", linestyle="-")
    ax.grid(True, which="minor", linestyle="--", alpha=0.2)
    ax.legend()

Now, let's compare the heat timeseries of different nuclear waste types:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
for model in models:
    q = model.heat(time, baseline=True)
    ax.loglog(time, q, label=model.name, lw=2.5)
format_ax(ax)
ax.set_ylim([1e-4, 5])
plt.show()


The bumps in the curves stem from the different leading nuclides in the 
proxy model sequentially decaying to nothing. The leading nuclides don't 
necessarily represent actual physical nuclides, but they give a close match
to the result of burn-off simulations.

In [ ]:
fig, axs = plt.subplots(
    nrows=int(0.5 + len(models) / 2), ncols=2, figsize=(16, 8), sharex=True
)
axs: list[plt.Axes] = np.reshape(axs, (-1))

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
for ax, model, color in zip(axs, models, colors):
    q = model.heat(time, baseline=True)
    ax.loglog(time, q, label=model.name, lw=2.5, c=color)
    for i in range(len(model.nuclide_powers)):
        q = model.heat(time, baseline=True, ncl_id=i)
        ax.loglog(time, q, label=f"Nuclide {i}", lw=1.5, c=color, ls=ls[i])
    format_ax(ax)
    ax.set_ylim([1e-4, 20])
plt.show()


We can create a heat model for a whole repository, by providing a proxy model,
a number of waste containers, the interim storage time and the time to fill
the repository. Let's compare a model which decays quickly (RK-BE + RK-HA) to
one which still releases a significant amount of heat after a million years 
(DWR-Mix).

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

be_ha_heat = repo.repo_be_ha_2016.heat(time)
dwr_heat = repo.repo_2020_conservative.heat(time)
ax.loglog(time, be_ha_heat, label="RK-BE + RK-HA", lw=2.5, c="red")
ax.loglog(time, dwr_heat, label="DWR-Mix", lw=2.5, c="green")
format_ax(ax)
ax.set_ylim([9, 25000])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 2))

ax.loglog(time, dwr_heat, label="DWR-Mix", lw=2.5, c="k")
for i in range(len(repo.repo_2020_conservative.waste[0].nuclide_powers)):
    q = repo.repo_2020_conservative.heat(time, ncl_id=i)
    ax.loglog(time, q, label=f"Nuclide {i}", lw=1.5, c="k", ls=ls[i])
format_ax(ax)
ax.set_ylim([9, 25000])
plt.show()
